In [8]:
from cheatsheet_utils import *
import pandas.io.formats.style
import imgkit
from PIL import ImageOps, Image
import os

options = {
  "enable-local-file-access": None
}

# Generate Top 12 brawlers overall
best_brawlers_df = get_best_brawlers()

def df_to_png(df, output_png):
    write_to_html_file(df, filename='temp.html')
    imgkit.from_file('temp.html', output_png, options=options)
    if output_png == 'infographics/infographics1.png':
        img = Image.open(output_png)
        border = (0, 0, 365, 0) # left, top, right, bottom
        img = ImageOps.crop(img, border)
        img.save(output_png)
        
    if output_png == 'infographics/checklist.png':
        img = Image.open(output_png)
        border = (0, 0, 620, 0) # left, top, right, bottom
        img = ImageOps.crop(img, border)
        img.save(output_png)
            
    os.remove('temp.html')
    print('Complete, image saved to:', output_png)
    
df_to_png(best_brawlers_df, 'infographics/infographics1.png')

# Generate best brawlers for each map. Save in 2 seperate png
cheatsheet = get_best_brawlers_map()
cheatsheet_part1 = cheatsheet.fillna('   ').head(9)
cheatsheet_part2 = cheatsheet.fillna('   ').tail(len(cheatsheet)-9)
df_to_png(cheatsheet_part1, 'infographics/infographics2.png')
df_to_png(cheatsheet_part2, 'infographics/infographics3.png')

checklist_df = get_best_brawlers_checklist(best_brawlers_df, 'c9')#, team_tags = team_tags_tribe)
df_to_png(checklist_df, 'infographics/checklist.png')

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Complete, image saved to: infographics/infographics1.png
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Complete, image saved to: infographics/infographics2.png
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Complete, image saved to: infographics/infographics3.png
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Complete, image saved to: infographics/checklist.png
